<a href="https://colab.research.google.com/github/cwscx/micrograd/blob/main/micrograd_from_scratch_yay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def f(x):
  return 3 * x ** 2 - 4 * x + 5  # 3x^2 - 4x + 5 => derivative: 6x - 4

In [ ]:
f(3.0)

In [ ]:
xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs, ys)

In [ ]:
h = 0.000000001
x = 2/3
(f(x + h) - f(x))/ h

In [ ]:
a = 2.0
b = -3.0
c = 10.0
d = a * b + c
print(d)

In [ ]:
h = 0.0001

#inputs = 2.0
a = 2.0
b = -3.0
c = 10.0

d1 = a * b + c
a += h
d2 = a * b + c

print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1) / h)

a = 2.0
d1 = a * b + c
b += h
d2 = a * b + c
print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1) / h)

b = -3.0
d1 = a * b + c
c += h
d2 = a * b + c
print('d1', d1)
print('d2', d2)
print('slope', (d2 - d1) / h)

In [ ]:
class Value:

  def __init__(self, data, _children=(), _op='', label='') -> None:
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self) -> str:
    return f"Value(data={self.data})"

  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    def _add_backward():
      self.grad += 1.0 * out.grad # gradient accumulates if refered multiple times
      other.grad += 1.0 * out.grad
    out._backward = _add_backward
    return out

  def __radd__(self, other):  ## to handle 1 + a
    return self + other

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    def _mul_backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _mul_backward
    return out

  def __rmul__(self, other):  ## to handle 1 * a
    return self * other

  def __neg__(self):
    return -1 * self

  def __sub__(self, other):
    return self + (-other)

  def __pow__(self, other):
    assert isinstance(other, (int, float))
    out = Value(self.data ** other, (self, ), f'**{other}')
    def _pow_backward():
      self.grad += out.grad * other * self.data ** (other - 1)
    out._backward = _pow_backward
    return out

  def __truediv__(self, other):
    return self * other ** -1


  def exp(self):
    n = self.data
    out = Value(math.exp(n), (self, ), 'exp')

    def _exp_backward():
      # d(e^x) / dx = e^x
      self.grad += out.grad * out.data

    out._backward = _exp_backward
    return out

  def tanh(self):
    n = self.data
    # derivative is 1- tanh(n)**2
    t = (math.exp(2*n) - 1) / (math.exp(2 * n) + 1)
    out = Value(t, (self, ), label='tanh')
    def _tanh_backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _tanh_backward
    return out

  def backward(self):
    # regular BFS won't work because this might not be a tree.
    topo = []
    visited = set()

    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)

    build_topo(self)
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [ ]:
a = Value(2.0, label='a')
b = Value(-3.0, label='b')
c = Value(10, label='c')
e = a * b
e.label = 'e'
d = e + c
d.label = 'd'
f = Value(-2.0, label='f')
L = d * f
L.label = 'L'
# L = ((a * b) + c) * f
print(d._prev, d._op)

In [ ]:
from graphviz import Digraph

def trace(root):
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format="svg", graph_attr={'rankdir': 'LR'})

  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))

    dot.node(name = uid, label = "{%s | data %.4f | grad %.4f}" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      dot.node(name = uid + n._op, label = n._op)
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [ ]:
draw_dot(L)

In [ ]:
plt.plot(np.arange(-5,5,0.2), np.tanh(np.arange(-5,5,0.2))); plt.grid()

In [ ]:
# inputs
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')

x1w1 = x1*w1
x1w1.label = 'x1*w1'
x2w2 = x2*w2
x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2
x1w1x2w2.label = 'x1w1 + x2w2'
n = x1w1x2w2 + b
n.label = 'n'
e = (2 * n).exp()
o = (e - 1) / (e + 1)
o.label = 'o'
o.backward()
draw_dot(o)

In [ ]:
draw_dot(o)

In [ ]:
o.grad = 1.0
o._backward()
n._backward()
b._backward()
x1w1x2w2._backward()
x1w1._backward()
x2w2._backward()

In [ ]:
o.grad = 1.0

# The order to backward propagation is reverse topological order.
# topological order: all children node is processed before the node
topo = []
visited = set()

def build_topo(v):
  if v not in visited:
    visited.add(v)
    for child in v._prev:
      build_topo(child)
    topo.append(v)

build_topo(o)

for node in reversed(topo):
  node._backward()

In [ ]:
o.backward()

In [ ]:
 a = Value(3.0, label='a')
 b = a + a;
 b.label = 'b'
 b.backward()
 draw_dot(b)

In [ ]:
a = Value(-2.8, label='a')
b = Value(3.0, label='b')
d = a * b
d.label = 'd'
e = a + b
e.label = 'e'
f = d * e
f.label = 'f'

f.backward()
draw_dot(f)

In [ ]:
a = Value(2.0)
b = Value(4.0)

a - b